In [18]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import linear_kernel

In [6]:
tfidf_dataset = pd.read_csv(r'C:/Users/mupsi/Desktop/crowd_curl_onlyme/NotGit/full_dataset_28jun.csv',dtype='string',index_col=False)

In [10]:
tfidf_dataset = tfidf_dataset.rename(columns = {'lemmatized_stopped':'lemm_tfidf_stopped'})

In [32]:
list(tfidf_dataset.columns)

['Unnamed: 0',
 'Link',
 'Raw_Text',
 'Hairtype',
 'Processed_Text',
 'lemmatized_tfidf',
 'lemm_tfidf_stopped']

In [99]:
tfidf = TfidfVectorizer(tokenizer=None, min_df = 200, ngram_range=(1,1))

In [90]:
def curl_query(product, tfidf_dataset, tfidf):
    """
    Return most similar based on cosine similarity
    
    """
    df_dum = pd.DataFrame([product], columns=['Processed_Text'])
    df_dum = pd.concat([tfidf_dataset, df_dum])
    query_vectors = tfidf.fit_transform(df_dum['Processed_Text'])
    cosine_similarities=linear_kernel(query_vectors[-1], query_vectors).flatten()
    related_docs = np.where((cosine_similarities>0.3) & (cosine_similarities<1))
    
    for i in related_docs:
        top_docs = []
        top_docs.extend(i)

    if len(top_docs) == 0:
        print("No results")
    else:
        top_output = df_dum.loc[top_docs]
        output = top_output[['Hairtype', 'Raw_Text']] #for datsaet after topic modeling:'First_Topic', 'Second_Topic', 'Third_Topic', 'Fourth_Topic']]
        output = output.sort_values(by=['Hairtype'])
        return output

In [101]:
product = 'cantu coconut'

In [102]:
curl_query(product, tfidf_dataset, tfidf)

,Hairtype,Raw_Text
25510,2a,"Cantu curl cream for wavies?Has anyone with 2a/2b/2c hair tried the Cantu Coconut Curls cream? How'd it work for you? I bought a travel sized one to try it out, which will probably last me quite a while because my hair is only about 4-5 inches long on top and shaved on the sides and back."
26207,2a,"Over 1 year CGM and not happy with results, can now afford better products though so I want to start experimenting2A/B, thick, high porosity, color treated, dry hair. My last post details my hair after a deep conditioning with coconut oil https://www.reddit.com/r/curlyhair/comments/edyfd8/day_after_deep_condition_and_my_hair_still/?utm_medium=android_app&amp;utm_source=share\n\nAnd this is today after refreshing and attempting the denman technique http://imgur.com/a/jzqACoB\n\nRoutine: alternate cowash with either As I am coconut co wash or Suave Essentials Coconut Conditioner 2X a week \n\nShampoo every 2 weeks with Not Your Mother's Blue Sea Kale Shampoo\n\nDeep condition every 2 weeks with coconut oil\n\nWhile wet praying hands to apply LA looks mega shaper gel and let air dry\n\nI've also tried the Cantu coconut curling cream and it was way too heavy\n\nWhat am I doing wrong?"
26027,2a,"2a/2b low porosity hair that's thick.I'm having a hard time finding the right products for my hair. The best thing I ever used was sunsilk waves of envy but that hasn't been sold in America in 10 years. \n\nPretty sure I have low porosity. My hair is frizzy, gets weighed down by things like Cantu cream, and a strand of my hair floats in water. \n\nI started off with using Cantu cowash, love beauty planet conditioner, squish to condish, cold rinse while upside down and squishing, microfiber scrunch, prayer hands Cantu cream, prayer hands LA gel, diffuse, scrunch the crunch and my hair looks limp and greasy. \n\nI've tried not doing Cantu cream, doing less, doing no gel, ect. But I still get limp waves."
25638,2a,"Best Cantu / Shea Moisture products for 2A / 2B hair?I’m currently using products from the beginner routine but they have an offer on at Superdrug for all curly products, including Cantu and Shea Moisture. \n\nNaturally there’s trial and error to find what works for me but I was wondering if anyone else with a similar hair type has had success with any of these products? \n\nI’ve got fine to medium low porosity hair that seems to get more defined each time I wash and style it. \n\nThere’s a Cantu Curl Activator, Cantu Curl Custard, Cantu Hold and Shine Mist, Shea Frizz Free Gel, and obviously the shampoos; just not sure where to start?! My main problems I guess are holding definition through the day, and some frizz (but that’s probably because I scrunch out the crunch too soon)"
28629,2b,"What do you think of the Cantu curl line?With all this Deva Curl madness I am properly paranoid now despite the fact that it has been my lifeline for the past 8 months. I am thinking about investing in the Cantu regular shampoo. I have high porosity, thick 2b-2c hair. Do you think it could work for me?"
...,...,...
38526,4c,"Question regarding daily use of leave in conditionerI'm really bad with understanding hair products and doing my hair because I'm a guy and always kept my hair short. so please bear with me\n\nHair type: 4B/4C Coily hair (Black people hair, not trying to be racist)\n\nRoutine : I use Cantu Argan Leave In Conditioner repair cream and will switch between the Cantu shea butter leave in repair cream daily. After the put that in I'll use either the Cantu coconut curling cream or the Cantu moisturizing curl activator. Every 2 days use the Cantu Curly Cream Rinse Out Conditioner. \n\nI notice with the leave in conditioner that it does what it says on the bottle and leaves moisturizing oils in my hair. Whenever I take a shower though, I can still feel it in my hair and I don't know if should completely rinse that stuff out or just leave some of it in? I need my hair really wet so that I can re-apply the leave in

### Tested edge case of "head and shoulders" through:
##### Processed_Text vs Lemmatized/Lemmatized_stopped text
##### min_df = 1 (default), 100, 200, 1000
##### ngram_range = (1,1) (default), (1,2), (1,3)
#### Best results come from: Processed_Text, 200, 1,1